In [3]:
#coding=utf-8
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt

In [4]:
mnist = input_data.read_data_sets('/media/lbk/ubuntu file/dataset/mnist/',one_hot=True)
trainimgs,trainlabels,testimgs,testlabels \
    =mnist.train.images,mnist.train.labels,mnist.test.images,mnist.test.labels

Extracting /media/lbk/ubuntu file/dataset/mnist/train-images-idx3-ubyte.gz
Extracting /media/lbk/ubuntu file/dataset/mnist/train-labels-idx1-ubyte.gz
Extracting /media/lbk/ubuntu file/dataset/mnist/t10k-images-idx3-ubyte.gz
Extracting /media/lbk/ubuntu file/dataset/mnist/t10k-labels-idx1-ubyte.gz


In [5]:
ntrain,ntest,dim,nclasses\
    = trainimgs.shape[0],testimgs.shape[0],trainimgs.shape[1],trainlabels.shape[1]

In [12]:
diminput = 28
dimhidden = 128
dimoutput = nclasses
nsteps = 28
# 不是一层里面的，对应两层
weights = {
    'hidden': tf.Variable(tf.random_normal([diminput,dimhidden])),
    'out': tf.Variable(tf.random_normal([dimhidden,dimoutput]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([dimhidden])),
    'out': tf.Variable(tf.random_normal([dimoutput]))
}

In [32]:
def _RNN(_X,_W,_b,_nsteps,_name):
    #[batch,h,w,1(c)] named [batch,nsteps,diminput]
    #==>[nsteps,batchsize,diminput]
    _X = tf.transpose(_X,[1,0,2])
    #==>[nsteps*batch,diminput]
    _X = tf.reshape(_X,[-1,diminput])
    #一个整体运算,相当于batch*w*h整体卷积
    _H = tf.matmul(_X,_W['hidden']) + _b['hidden']
    #切分乘nestep个序列
    _Hsplit = tf.split(_H,_nsteps,0)
    #lstm
    with tf.variable_scope(_name) as scope:
        scope.reuse_variables()
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(dimhidden,forget_bias=1.0)
        #
        _LSTM_O,_LSTM_S = tf.nn.static_rnn(lstm_cell,_Hsplit,dtype=tf.float32)
    #output
    _O = tf.matmul(_LSTM_O[-1],_W['out']) + _b['out']
    return {
        'X':_X,'H':_H,'Hsplit':_Hsplit,'LSTM_O':_LSTM_O,'LSTM_S':_LSTM_S,'O':_O
    }
print 'ready'

ready


In [34]:
learning_rate = 0.001
x = tf.placeholder('float',[None,nsteps,diminput])
y = tf.placeholder('float',[None,dimoutput])

myrnn = _RNN(x,weights,biases,nsteps,'basic')
pred = myrnn['O']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred,labels = y))
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1),tf.argmax(y,1)),tf.float32))

init = tf.global_variables_initializer()


In [35]:
training_epochs = 5
batch_size = 16
display_step = 1
sess = tf.Session()
sess.run(init)

print 'start'

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = 100
    for i in range(total_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size,nsteps,diminput))
        
        feeds = {x:batch_xs,y:batch_ys}
        sess.run(optm,feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)/total_batch
        
    print('epoch:%03d/%03d cost:%.9f'%(epoch,training_epochs,avg_cost))

start
epoch:000/005 cost:1.629186481
epoch:001/005 cost:1.147092171
epoch:002/005 cost:0.949910490
epoch:003/005 cost:0.867408370
epoch:004/005 cost:0.776862026
